Due to strong, probably-magical winds, the hailstones are all flying through the air in perfectly linear trajectories. You make a note of each hailstone's position and velocity (your puzzle input). Each line of text corresponds to the position and velocity of a single hailstone. The positions indicate where the hailstones are right now (at time 0). The velocities are constant and indicate exactly how far each hailstone will move in one nanosecond. Each line of text uses the format px py pz @ vx vy vz. For instance, the hailstone specified by 20, 19, 15 @ 1, -5, -3 has initial X position 20, Y position 19, Z position 15, X velocity 1, Y velocity -5, and Z velocity -3. After one nanosecond, the hailstone would be at 21, 14, 12.

Perhaps you won't have to do anything. How likely are the hailstones to collide with each other and smash into tiny ice crystals? To estimate this, consider only the X and Y axes; ignore the Z axis. Looking forward in time, how many of the hailstones' paths will intersect within a test area? (The hailstones themselves don't have to collide, just test for intersections between the paths they will trace.)

However, you'll need to search a much larger test area if you want to see if any hailstones might collide. Look for intersections that happen with an X and Y position each at least 200000000000000 and at most 400000000000000. Disregard the Z axis entirely.

Considering only the X and Y axes, check all pairs of hailstones' future paths for intersections. How many of these intersections occur within the test area?

In [28]:
# 2e14 - 4e14
# y = mx + b
# (x, y) = (stone[0], stone[1])
# m = stone[4]/stone[3]

stones = [list(map(int, line.replace('@', ',').split(','))) for line in open('input.txt')]
lo, hi = 2e14, 4e14

def intersect(a, b):
    ma = a[4]/a[3]
    ca = a[1] - a[0]*ma
    
    mb = b[4]/b[3]
    cb = b[1] - b[0]*mb    

    if ma == mb:
        return False, (0, 0), (0, 0)
    else:
        x = (cb - ca)/(ma - mb)
        y = ma*x + ca

        ta = (x - a[0])/a[3]
        tb = (x - b[0])/b[3]

        return True, (x, y), (ta, tb)

count = 0
for i in range(len(stones)-1):
    for j in range(i+1, len(stones)):
        crosses, pos, time = intersect(stones[i], stones[j])
        if crosses:
            if lo <= pos[0] <= hi and lo <= pos[1] <= hi and time[0] > 0 and time[1] > 0:
                count += 1

print(count)

17906


You can use the probably-magical winds to reach any integer position you like and to propel the rock at any integer velocity. Now including the Z axis in your calculations, if you throw the rock at time 0, where do you need to be so that the rock perfectly collides with every hailstone? Due to probably-magical inertia, the rock won't slow down or change direction when it collides with a hailstone.

In [1]:
import sympy

stones = [tuple(map(int, line.replace("@", ",").split(","))) for line in open('input.txt')]

xr, yr, zr, vxr, vyr, vzr = sympy.symbols("xr, yr, zr, vxr, vyr, vzr")

equations = []

# sx + t*vx = xr + t*vxr --> t = (xr - sx)/(vx - vxr)
# sy + t*vy = yr + t*vyr --> t = (yr - sy)/(vy - vyr)
# sz + t*vz = zr + t*vzr --> t = (zr - sz)/(vz - vzr)

# (xr - sx)/(vx - vxr) = (yr - sy)/(vy - vyr) = (zr - sz)/(vz - vzr)
# exp a: (xr - sx)*(vy - vyr) - (yr - sy)*(vx - vxr) = 0
# exp b: (yr - sy)*(vz - vzr) - (zr - sz)*(vy - vyr) = 0

for i, (sx, sy, sz, vx, vy, vz) in enumerate(stones):
    equations.append((xr - sx) * (vy - vyr) - (yr - sy) * (vx - vxr))
    equations.append((yr - sy) * (vz - vzr) - (zr - sz) * (vy - vyr))
    if i < 2:
        continue
    answers = [soln for soln in sympy.solve(equations) if all(x % 1 == 0 for x in soln.values())]
    if len(answers) == 1:
        break
    
answer = answers[0]

print(answer[xr] + answer[yr] + answer[zr])
print(i)

571093786416929
2
